<a href="https://colab.research.google.com/github/JohnVarkey/Deep-Learning/blob/master/Conv3d%20Model%2060%25%20acc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!pip install kaggle

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset -p /content

100% 3.58G/3.58G [01:05<00:00, 54.6MB/s]
100% 3.58G/3.58G [01:05<00:00, 58.7MB/s]


In [4]:
!unzip \*.zip

Archive:  real-life-violence-situations-dataset.zip
  inflating: Real Life Violence Dataset/NonViolence/NV_1.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_10.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_100.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_1000.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_101.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_102.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_103.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_104.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_105.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_106.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_107.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_108.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_109.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_11.mp4  
  inflating: Real Life Violence Dataset/N

In [5]:
!mkdir TrainData && cd TrainData && mkdir NonViolence && mkdir Violence && echo done
!mkdir TestData && cd TestData && mkdir NonViolence && mkdir Violence && echo done

done
done


In [6]:
!rm -r real-life-violence-situations-dataset.zip && echo done
 

done


In [7]:
import glob
from tqdm import tqdm
import cv2
import math


globpath=sorted(glob.glob("/content/Real Life Violence Dataset/*/*"))
skipcount=0
namecount=0
for vidfile in tqdm(globpath):
  filename=vidfile.split("/")[-2]
  if vidfile=="/content/Real Life Violence Dataset/NonViolence/NV_1.mp4" or vidfile=="/content/Real Life Violence Dataset/Violence/V_1.mp4":
     namecount=1
  vid=cv2.VideoCapture(vidfile)
  fps = vid.get(cv2.CAP_PROP_FPS)
  fps_rate=math.floor(fps/3)
  while True:
    ret,frame=vid.read()
    frameId = vid.get(1)
    if not ret:
      break
    if (frameId % math.floor(fps) != 0 or skipcount<fps_rate):
        skipcount+=1
        continue
    skipcount=0
    if namecount<1249:
      cv2.imwrite("/content/TrainData/{}/{}_{}.jpg".format(filename,filename[0],namecount),frame)
    else:
      cv2.imwrite("/content/TestData/{}/{}_{}.jpg".format(filename,filename[0],namecount),frame)
    
    namecount+=1
  vid.release()

print(namecount)

100%|██████████| 2000/2000 [11:02<00:00,  3.02it/s]

5119


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
from tweaked_ImageGenerator_v2 import ImageDataGenerator
datagen = ImageDataGenerator()
!mkdir chkp
input_Shape=(5,64,64,3)
test_data=datagen.flow_from_directory('/content/TrainData/', target_size=input_Shape[1:3], batch_size=10, frames_per_step=input_Shape[0])
train_data=datagen.flow_from_directory('/content/TestData/', target_size=input_Shape[1:3], batch_size=10, frames_per_step=input_Shape[0])




Using TensorFlow backend.


Found 2496 images belonging to 2 classes.
Found 7619 images belonging to 2 classes.


In [ ]:

import keras
from keras.layers import Dense,Dropout
from keras.utils.vis_utils import plot_model 

layers = keras.layers
models = keras.models
losses = keras.losses
optimizers = keras.optimizers
metrics = keras.metrics
num_classes = 2
input_shapes = (100,100,3)
vg19 = keras.applications.vgg19.VGG19
base_model = vg19(include_top=False,input_shape=(100,100,3))
for layer in base_model.layers:
    layer.trainable = False
model = models.Sequential()
num_classes = 2
cnn = models.Sequential()
cnn.add(base_model)
cnn.add(layers.Flatten())
model = models.Sequential()
model.add(layers.TimeDistributed(cnn,  input_shape=(4, 100, 100, 3)))
model.add(layers.LSTM(40))
model.add(layers.Dense(13, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(num_classes, activation="sigmoid"))
adam = optimizers.Adam(0.001)

adam = optimizers.Adam(0.001)

model.compile(loss='binary_crossentropy', optimizer= adam, metrics=["accuracy"])
model.summary()

Using TensorFlow backend.


80142336/80134624 [==============================] - 7s 0us/step
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 4, 4608)           20024384  
_________________________________________________________________
lstm_1 (LSTM)                (None, 40)                743840    
_________________________________________________________________
dense_1 (Dense)              (None, 13)                533       
_________________________________________________________________
dropout_1 (Dropout)          (None, 13)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 28        
Total params: 20,768,785
Trainable params: 744,401
Non-trainable params: 20,024,384
_________________________________________________________________


In [ ]:
#accracy 57%
import keras
from keras.layers import Dense,Dropout
from keras.utils.vis_utils import plot_model 

layers = keras.layers
models = keras.models
losses = keras.losses
optimizers = keras.optimizers
metrics = keras.metrics


model=models.Sequential()
#model.add(layers.TimeDistributed(cnn,  input_shape=input_Shape))
model.add(layers.Conv3D(96,(1,8,8),strides=(1,3,3), input_shape=input_Shape))
model.add(layers.MaxPool3D(pool_size=(1,5,5), strides=(1,2,2)))
model.add(layers.Conv3D(384,(2,4,4)))
model.add(layers.Conv3D(200,(3,3,3),strides=(1,1,1)))
model.add(layers.MaxPool3D(pool_size=(1,2,2)))
model.add(layers.Conv3D(200,(1,3,3),strides=(1,1,1)))
model.add(layers.MaxPool3D(pool_size=(1,2,2)))
model.add(layers.TimeDistributed(layers.Flatten()))
model.add(layers.LSTM(1000, return_sequences=False))


#model.add(layers.LSTM(100))


#model.add(EarlyStopping(monitor='val_loss', patience=2))


model.add(Dense(1000, activation='relu'))
model.add(Dropout(.50))
model.add(Dense(250, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

adam = optimizers.Adam(0.001)

model.compile(loss='binary_crossentropy', optimizer= adam, metrics=["accuracy"])
model.summary()

In [10]:

import keras
from keras.layers import LSTM,TimeDistributed, Input,Conv3D, MaxPooling3D, Flatten, Dense,Dropout, Activation
from keras.models import Sequential
from keras.optimizers import Adam


model = Sequential()

model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=input_Shape, padding='same'))


model.add(Activation('relu'))
model.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('softmax'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(Dropout(0.25))
model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('softmax'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(512))
model.add(Dropout(0.25))
#model.add(Flatten())
model.add(Dense(512, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

#model.load_weights('/content/drive/My Drive/training/chkp/conv3dlitev2.1_weights.33-0.70.hdf5')
adam = Adam(0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=adam, metrics=['accuracy'])
model.summary()






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 5, 64, 64, 32)     2624      
_________________________________________________________________
activation_1 (Activation)    (None, 5, 64, 64, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 5, 64, 64, 32)     27680     
_________________________________________________________________
activation_2 (Activation)    (None, 5, 64, 64, 32)     0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 2, 22, 22, 32)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 22, 22, 32)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 2, 22, 22, 64)    

In [14]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        '/content/chkp/conv3dlstm_weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
]
history=model.fit_generator(
    train_data,
    validation_data=test_data,
    verbose=1,
    steps_per_epoch=761,
    epochs=70,
    validation_steps=249,
    callbacks=callbacks
)

Epoch 1/70
761/761 [==============================] - 539s 709ms/step - loss: 0.6820 - accuracy: 0.5193 - val_loss: 0.7053 - val_accuracy: 0.5116

Epoch 00001: saving model to /content/chkp/conv3dlstm_weights.01-0.71.hdf5
Epoch 2/70
761/761 [==============================] - 533s 700ms/step - loss: 0.6677 - accuracy: 0.5373 - val_loss: 0.6919 - val_accuracy: 0.5104

Epoch 00002: saving model to /content/chkp/conv3dlstm_weights.02-0.69.hdf5
Epoch 3/70
761/761 [==============================] - 526s 691ms/step - loss: 0.6652 - accuracy: 0.5403 - val_loss: 0.7009 - val_accuracy: 0.5173

Epoch 00003: saving model to /content/chkp/conv3dlstm_weights.03-0.70.hdf5
Epoch 4/70
761/761 [==============================] - 532s 699ms/step - loss: 0.6569 - accuracy: 0.5432 - val_loss: 0.6795 - val_accuracy: 0.5159

Epoch 00004: saving model to /content/chkp/conv3dlstm_weights.04-0.68.hdf5
Epoch 5/70
761/761 [==============================] - 528s 694ms/step - loss: 0.6556 - accuracy: 0.5479 - val_lo

KeyboardInterrupt: ignored